# **Object Detection - Models Training**

---

**Mateus Aleixo - 124256**

---

In [9]:
from collections import Counter
from glob import glob
import os
import random
from rfdetr import RFDETRBase
import shutil
import torch
from ultralytics import YOLO

In [10]:
def get_image_label_pairs(
    image_root, annotation_root, image_ext="jpg", label_ext="txt"
):
    pairs = []
    image_files = glob(os.path.join(image_root, "**", f"*.{image_ext}"), recursive=True)

    for img in image_files:
        rel_path = os.path.relpath(img, image_root)
        label = os.path.join(
            annotation_root, os.path.splitext(rel_path)[0] + f".{label_ext}"
        )

        if os.path.exists(label):
            pairs.append((img, label))
        else:
            print(f"Warning: Label file not found for image {img}")

    return pairs

In [11]:
image_root = "../data/augmented_data/frames"
annotation_root = "../data/annotations"
pairs = get_image_label_pairs(image_root, annotation_root)
print(f"Found {len(pairs)} image-label pairs.")

Found 3864 image-label pairs.


In [12]:
empty_files = []

for root, _, files in os.walk(annotation_root):
    for file in files:
        file_path = os.path.join(root, file)

        if os.stat(file_path).st_size == 0:
            empty_files.append(file_path)

print(f"Found {len(empty_files)} empty label files.")

Found 0 empty label files.


In [13]:
label_counts = Counter()

for root, _, files in os.walk(annotation_root):
    for file in files:
        with open(os.path.join(root, file), "r") as f:
            lines = f.readlines()

            for line in lines:
                class_id = line.split()[0]
                label_counts[class_id] += 1

label_counts

Counter({'1': 7728, '2': 7728, '3': 7728, '0': 7726, '4': 7694})

In [14]:
# Paths
image_dir = "../data/augmented_data/frames"
label_dir = "../data/annotations"
output_dir = "../data/split_data"

# Create output folders
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

# Get all images
all_images = []
for root, _, files in os.walk(image_dir):
    for file in files:
        if file.endswith(".jpg") or file.endswith(".png"):
            all_images.append(os.path.join(root, file))

# Shuffle and split
random.shuffle(all_images)
train_ratio = 0.7
val_ratio = 0.2
train_count = int(len(all_images) * train_ratio)
val_count = int(len(all_images) * val_ratio)

train_images = all_images[:train_count]
val_images = all_images[train_count : train_count + val_count]
test_images = all_images[train_count + val_count :]

# Move files
def move_files(image_list, split):
    for img_path in image_list:
        # Get label path
        rel_path = os.path.relpath(img_path, image_dir)
        label_path = os.path.join(label_dir, os.path.splitext(rel_path)[0] + ".txt")

        # Define new paths
        new_img_path = os.path.join(
            output_dir, "images", split, os.path.basename(img_path)
        )
        new_label_path = os.path.join(
            output_dir, "labels", split, os.path.basename(label_path)
        )

        # Copy files
        shutil.copy(img_path, new_img_path)
        if os.path.exists(label_path):
            shutil.copy(label_path, new_label_path)


# Move train, val & test files
move_files(train_images, "train")
move_files(val_images, "val")
move_files(test_images, "test")

print(
    f"Dataset split completed: {len(train_images)} train, {len(val_images)} val, {len(test_images)} test"
)

Dataset split completed: 2704 train, 772 val, 388 test


In [15]:
yolo = YOLO(verbose=False)

results = yolo.train(
    data="../data/dataset.yaml",
    epochs=22,
    batch=8,
    imgsz=416,
    device="cuda" if torch.cuda.is_available() else "cpu",
    optimizer="AdamW",
    lr0=0.001,
    lrf=0.01,
    weight_decay=0.0005,
    patience=3,
    degrees=10,
    translate=0.2,
    scale=0.5,
    shear=2.0,
    perspective=0.001,
    flipud=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.2,
)

New https://pypi.org/project/ultralytics/8.3.102 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.40  Python-3.12.9 torch-2.6.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=../data/dataset.yaml, epochs=22, time=None, patience=3, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fra

train: Scanning C:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\data\split_data\labels\train... 1918 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1918/1918 [00:02<00:00, 714.04it/s]

train: New cache created: C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\data\split_data\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning C:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\data\split_data\labels\val... 710 images, 0 backgrounds, 0 corrupt: 100%|██████████| 710/710 [00:00<00:00, 788.97it/s]

val: New cache created: C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\data\split_data\labels\val.cache


...

      22/22         0G     0.9891     0.4836     0.8129         60        416: 100%|██████████| 240/240 [02:25<00:00,  1.65it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:19<00:00,  2.29it/s]

                   all        710       7084      0.988      0.991       0.99      0.786



22 epochs completed in 1.272 hours.
Optimizer stripped from c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train\weights\last.pt, 5.5MB
Optimizer stripped from c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train\weights\best.pt, 5.5MB

Validating c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train\weights\best.pt...
Ultralytics 8.3.40  Python-3.12.9 torch-2.6.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
YOLO11n summary (fused): 238 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:17<00:00,  2.52it/s]


                   all        710       7084      0.988      0.991       0.99      0.786
                  blue        710       1420      0.998      0.998      0.994      0.817
                 green        710       1420      0.983      0.984      0.989      0.746
                orange        710       1420      0.988      0.989      0.989      0.794
                   red        710       1420      0.991      0.991      0.992      0.793
                yellow        702       1404      0.979      0.993      0.988      0.781
Speed: 0.4ms preprocess, 17.9ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train
